# Examples of Variable Expressions in AllVue

#### Key things to remember throughout

* If the expression throws an error for *any* respondent, the whole chart will error
* Only a subset of python is supported in AllVue, so if an expression that works here is rejected, check the [docs](https://docs.savanta.com/internal/Content/AllVue/Introduction_to_Field_Expressions.html?Highlight=expressions), and ask Aila for an equivalent expression that only uses syntax mentioned in the [docs](https://docs.savanta.com/internal/Content/AllVue/Introduction_to_Field_Expressions.html?Highlight=expressions)

#### Approach

Write down these three things in text form:
1. **Input shape** - Is there one result? Or is it split down per entity?
2. **Result shape** - Is there one result? Or is it split down per entity?
    * In a UI world this would be the props
    * In an Object Oriented world this would be the fields of the class
    * In a functional world this would be the parameters
3. **Transformation requirements**

4. In Python or tabular format, write down one example of input data.
5. Write down the expected output for this input data.

Now build up the expression piece by piece to get there

#### Python objects available

Expressions are always evaluated in the context of a single respondent, and a single result data point.

* `response.some_variable()` returns a list of the values in the "Value" column of some_variable for this respondent 
  * `response.some_variable(some_entity=[7]})` returns the above list, but only returns the "Value" where the "some_entity" column matches the given value or values
* If (and only if) you mention result.some_entity, the output data will have a column called some_entity.
  * `result.some_entity` is always an integer id of the entity on the result row currently being considered.
  * If you want the result to have a row for instance of some_entity, you must mention result.some_entity

Note: By convention, for single-answer questions/variables, the Value is equal to the answer entity id

#### Import all the bits we'll need to simulate AllVue's expression evaluation

In [156]:
from allvue_simulator import Response, Result, QuestionVariable, ExpressionVariable, evaluate

## Example 1: Age

* Input shape: No entities
* Output shape: No entities
* Transformation requirements: None - just a syntax example

### Setup a single respondent's answers

In [157]:
response = Response()
response.age = QuestionVariable(
    ["Value"], # No entity types here, just the value which is always last
    [
        [25], # One row of data with age
    ]
)

In [158]:
result = Result() # AllVue loops over each result data point on a chart - we'll just pick one to test with here.

In [159]:
response.age() # Not a valid expression for AllVue since it returns a list - see the square brackets below?


[25]

In [160]:
max(response.age())

25

## Example 2: Categorical scale into value

* Input shape: In the survey Q4NEW is a categorical scale (Q4NEWAnswers) with 10 possible values as well as 97 for "I don't know"
* Output shape: The variable's result should have no entities
  * Therefore, we won't mention "result.someentity"
* Transformation requirements: The numeric value of the answer, or None if the answer is 97

In [161]:
response = Response()
response.Q4NEW = QuestionVariable(
    ["Q4NEWAnswers", "Value"],
    [
        [8, 8] # Change these - singlecode question has entity and value the same
    ]) 

result = Result() # result has no dimensions

# Filtering by entity solution
max(response.Q4NEW(Q4NEWAnswers=[1,2,3,4,5,6,7,8,9,10]), default = None)

8

In [162]:
# Alternative: Filtering value with ternary conditional
None if max(response.Q4NEW(), default = None) == 97 else max(response.Q4NEW(), default = None)

8

In [163]:
# Alternative: Filtering value with list comprehension
max([a for a in response.Q4NEW() if a != 97], default = None)

8

Make sure you've tried changing the response data above to be a scale value between 1 and 10, and 97 to check it returns None

## Example 3: Min value

https://morar.freshdesk.com/a/tickets/15765



* Input shape: "frequently_platform_week_QP604" question has two entities: Q603Answers (social media platforms) and Q604Answers (frequency of use) going from 1=Most to 4=Least
* Output shape: Variable needs to have the 5 options from the QP604Answers entity set
  * Therefore we must use result.Q604Answers
* Transformation requirements: The value for each instance should only be set for the highest frequency of use (i.e. lowest value) for any of the social media platforms they were asked about (QP603Answers).

In [164]:
# Input shape

response = Response()
response.frequently_platform_week_QP604 = QuestionVariable(
    ["Q603Answers", "Q604Answers", "Value"],
    [ # This respondent has 3 answers to the question
     # This is a radio button question, so the value is equal to the answer they clicked from Q604Answers (frequency of use)
        [8,  2, 2], #Q603Answers=8, Q604Answers=2. e.g. "I use facebook three times a day"
        [9,  2, 2],
        [10, 3, 3]
    ])

Output shape for individual user

| Q604Answers | Value  |
|-            | -      |
|1            | None   |
|2            | 2      |
|3            | None   |
|4            | None   |
|5            | None   |

This acts like a radio button question, as if they just selected "I use some social media three times a day"
Note that the None values are the same as omitting the row entirely, just to help consider those cases

All this means if (and only if) the Result has Q604Answers set to 2, our expression should return 2, otherwise it should return None

In [165]:
result = Result(Q604Answers=3) # Setup which result data point we're currently looking at, change the number to test different data points

# Below, test out what some of the bits we'll need look like before putting together

#### Exercise: Try to solve this example HERE before scrolling down

In [166]:

# Build up piece by piece, e.g.
result.Q604Answers # The entity id for the result data point we're currently considering - this must be part of the expression

3

In [167]:
response.frequently_platform_week_QP604() # e.g. All the values for this respondent + question

[2, 2, 3]

### Do you working above this cell, putting things together. Hints and answers are below
```







```

In [168]:
# Get the value for their most used platform (the scale goes from most to least used, so we actually want the minimum!)
min(response.frequently_platform_week_QP604(), default=None)

2

In [169]:
# But we only to know if this respondent should be included in the current result, i.e. is their minimum equal to the result's entity
min(response.frequently_platform_week_QP604(), default=None) == result.Q604Answers


False

In [170]:

# The above could be a base expression, but we actually want to output the value so it can be used for things like filter value mapping
result.Q604Answers if min(response.frequently_platform_week_QP604(), default=None) == result.Q604Answers else None

Now try switching the result to be for frequency of 3 and rerun to see the output for that result should be None (which won't print anything)

Further examples:
* Combining many variables: https://morar.freshdesk.com/a/tickets/15779

### Example 4: Combining variables in BrandVue!

Figure out the expression here, then in BrandVue 360 on test create a new variable called Test_{YourInitials}_{WhateverYouLike}:
https://savanta.test.all-vue.com/brandvue/ui/metric-configuration?Active=1
Here's the survey if it helps: https://fieldvue.savanta.com/SurveyLive?uid=7497223a0b


#### Part 1: Most recent job

The dashboard currently contains indicate_chief_earner_SEG1 and indicate_chief_earner_SEG2

SEG2 is only populated if they say retired for SEG1 - we ask what job they had before retiring

Create a new variable MostRecentJob with a metric that shows the percentage of people for each most recent job (i.e. for retired people it should pull through SEG2), the rest should just be SEG1

#### Part 2: Most recent job for visitors to brand

I want the same as above, but I want the user to be able to see the breakdown for a particular brand's users
i.e. I want the output shape to have the Job dimension and the brand dimension

Visitors to a brand are captured by the Consumer_Segment question - people who answered 3 or 4
